# 第6回講義 宿題

## 課題

CNNを用いて、FashionMNISTの高精度な分類器を実装してみましょう。

モデルのレイヤーを変更してみるなどして精度の向上にチャンレンジして下さい。 精度上位者はリーダーボードに載ります。

### 目標値

Accuracy 93%

### ルール

- 訓練データはx_train、 t_train、テストデータはx_testで与えられます。
- 予測ラベルは one_hot表現ではなく0~9のクラスラベル で表してください。
- **下のセルで指定されているx_train、t_train以外の学習データは使わないでください。**
- Tensorflowを利用して構いません。
- ただし、**tf.layersのような高レベルのAPIは使用しないで下さい。**具体的に以下のモジュールは使用しないでください。

```
tf.app,
tf.compat,
tf.contrib,
tf.estimator,
tf.gfile,
tf.graph_util,
tf.image,
tf.initializers,
tf.keras,
tf.layers,
tf.logging,
tf.losses,
tf.metrics,
tf.python_io,
tf.resource_loader,
tf.saved_model,
tf.sets,
tf.summary,
tf.sysconfig,
tf.test
```

### 提出方法

- 2つのファイルを提出していただきます。
  - テストデータ (x_test) に対する予測ラベルをcsvファイル (ファイル名: submission_pred.csv) で提出してください。
  - それに対応するpythonのコードをsubmission_code.pyとして提出してください (%%writefileコマンドなどを利用してください)。

### 評価方法

- 予測ラベルのt_testに対する精度 (Accuracy) で評価します。
- 毎日夜24時にテストデータの一部に対する精度でLeader Boardを更新します。
- 締切日の夜24時にテストデータ全体に対する精度でLeader Boardを更新します。これを最終的な評価とします。

### データの読み込み

- この部分は修正しないでください

In [1]:
!pip3 install tensorflow-gpu==1.8

You are using pip version 8.1.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [18]:
import numpy as np
import pandas as pd
import tensorflow as tf

try:
    del [
        tf.app,
        tf.compat,
        tf.contrib,
        tf.estimator,
        tf.gfile,
        tf.graph_util,
        tf.image,
        tf.initializers,
        tf.keras,
        tf.layers,
        tf.logging,
        tf.losses,
        tf.metrics,
        tf.python_io,
        tf.resource_loader,
        tf.saved_model,
        tf.sets,
        tf.summary,
        tf.sysconfig,
        tf.test
    ]
    
except AttributeError:
    print('Unrequired modules are already deleted (Skipped).')

def load_mnist():

    # 学習データ
    x_train = np.load('/root/userspace/public/chap06/data/x_train.npy')
    t_train = np.load('/root/userspace/public/chap06/data/t_train.npy')
    
    # テストデータ
    x_test = np.load('/root/userspace/public/chap06/data/x_test.npy')

    x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255
    x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255
    t_train = np.eye(10)[t_train.astype('int32').flatten()]

    return (x_train, x_test, t_train)

Unrequired modules are already deleted (Skipped).


### 畳み込みニューラルネットワーク(CNN)の実装

In [50]:
%%writefile /root/userspace/chap06/materials/submission_code.py

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

rng = np.random.RandomState(1234)
random_state = 8

### レイヤー定義 ###

class Conv:
    # WRITE ME
    def __init__(self, filter_shape, function=lambda x: x, strides=[1,1,1,1], padding='VALID'):
        # Heの初期値
        fan_in = np.prod(filter_shape[:3]) # filter_shape: (縦の次元数)x(横の次元数)x(入力チャンネル数)x(出力チャンネル数)
        fan_out = np.prod(filter_shape[:2]) * filter_shape[3]
        self.W = tf.Variable(rng.uniform(
                        low=-np.sqrt(6/fan_in),
                        high=np.sqrt(6/fan_in),
                        size=filter_shape
                    ).astype('float32'), name='W')
        self.b = tf.Variable(np.zeros((filter_shape[3]), dtype='float32'), name='b') # バイアスはフィルタごとなので, 出力フィルタ数と同じ次元数
        self.function = function
        self.strides = strides
        self.padding = padding

    def __call__(self, x):
        u = tf.nn.conv2d(x, self.W, strides=self.strides, padding=self.padding) + self.b
        return self.function(u)    
    
class Pooling:
    # WRITE ME
    def __init__(self, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID'):
        self.ksize = ksize
        self.strides = strides
        self.padding = padding
    
    def __call__(self, x):
        return tf.nn.avg_pool(x, ksize=self.ksize, strides=self.strides, padding=self.padding) #maxからaverageに変更
    
class Flatten:
    # WRITE ME
    def __call__(self, x):
        return tf.reshape(x, (-1, np.prod(x.get_shape().as_list()[1:])))
    
class Dense:
    # WRITE ME
    def __init__(self, in_dim, out_dim, function=lambda x: x):
        # He Initialization
        # in_dim: 入力の次元数、out_dim: 出力の次元数
        self.W = tf.Variable(rng.uniform(
                        low=-np.sqrt(6/in_dim),
                        high=np.sqrt(6/in_dim),
                        size=(in_dim, out_dim)
                    ).astype('float32'), name='W')
        self.b = tf.Variable(np.zeros([out_dim]).astype('float32'))
        self.function = function

    def __call__(self, x):
        return self.function(tf.matmul(x, self.W) + self.b)
    
def tf_log(x):
    # WRITE ME
    return tf.log(tf.clip_by_value(x, 1e-10, x))
    
### ネットワーク ###

x_train, x_test, t_train = load_mnist()
x_train, x_valid, t_train, t_valid = train_test_split(x_train, t_train, test_size=0.1, random_state=random_state)

tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 28, 28, 1])
t = tf.placeholder(tf.float32, [None, 10])

# WRITE ME 計算グラフの構築
                                             # (縦の次元数)x(横の次元数)x(チャネル数)
# h = Conv((5, 5, 1, 20), tf.nn.relu)(x)           # 28x28x 1 -> 24x24x20
# h = Pooling((1, 2, 2, 1))(h)                           # 24x24x20 -> 12x12x20
# h = Conv((5, 5, 20, 50), tf.nn.relu)(h)        # 12x12x20 ->  8x 8x50
# h = Pooling((1, 2, 2, 1))(h)                           #  8x 8x50 ->  4x 4x50
# h = Flatten()(h)
# y = Dense(4*4*50, 10, tf.nn.softmax)(h)

#上に代わってこちらを追加
# layers = [
#     Conv((5, 5, 1, 20), tf.nn.relu),
#     Pooling((1, 2, 2, 1)),
#     Conv((5, 5, 20, 50), tf.nn.relu),
#     Pooling((1, 2, 2, 1)),
#     Flatten(),
#     Dense(4*4*50, 10, tf.nn.softmax)
# ]
#試し用
layers = [
    Conv((5, 5, 1, 20), tf.nn.relu, padding='VALID'),
    Conv((5, 5, 20, 20), tf.nn.relu, padding='SAME'),
    Pooling((1, 2, 2, 1)),
    Conv((5, 5, 20, 50), tf.nn.relu, padding='VALID'),
    Pooling((1, 2, 2, 1)),
    Flatten(),
    Dense(4*4*50, 10, tf.nn.softmax)
]

def f_props(layers, h):
    for layer in layers:
        h = layer(h)
    return h
y = f_props(layers, x)

cost = - tf.reduce_mean(tf.reduce_sum(t * tf_log(y), axis=1))
train = tf.train.AdamOptimizer(0.001).minimize(cost) #optimizerはGradientDescentから適宜変える

### 学習 ###

n_epochs = 25
batch_size = 100
n_batches = x_train.shape[0]//batch_size

init = tf.global_variables_initializer()

# with tf.Session() as sess:
    # WRITE ME
sess = tf.Session() #error回避のためこっちに変えた。
sess.run(init)
for epoch in range(n_epochs):
    x_train, t_train = shuffle(x_train, t_train, random_state=random_state)
    for batch in range(n_batches):
        start = batch * batch_size
        end = start + batch_size
        sess.run(train, feed_dict={x: x_train[start:end], t: t_train[start:end]})
    y_pred, cost_valid = sess.run([y, cost], feed_dict={x: x_valid, t: t_valid})
    print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(
        epoch,
        cost_valid,
        accuracy_score(t_valid.argmax(axis=1), y_pred.argmax(axis=1))
    ))
    
# WRITE ME
y_pred = sess.run(y, feed_dict={x: x_test})
y_pred = y_pred.argmax(axis=1)
print(y_pred)

submission = pd.Series(y_pred, name='label')
submission.to_csv('/root/userspace/chap06/materials/submission_pred.csv', header=True, index_label='id')

Writing /root/userspace/chap06/materials/submission_code.py
